In [201]:
#Load 131k seq data 

from Bio import SeqIO
import pandas as pd

eyed = []
name = []
desc = []

for seq_record in SeqIO.parse('Complete NCBI Virus Genbank only.fasta', "fasta"):
    eyed.append(seq_record.id)
    desc.append(seq_record.description.lower())

#All following anomalies in this cell had species name missing
#Fix anomaly at index 21244

desc[21244] = 'MK293717 |Gemycircularvirus strain RN/Haikou/China| complete genome|complete|China|Rattus norvegicus|Gemycircularvirus'.lower()

#Fix anomaly at index 40853

desc[40853] = 'mf407276 |bacillus phage vb_bvep-goe6| complete genome|complete||bacillus velezensis fzb42|bacillus phage'

#Fix anomaly at index 50896

desc[50896] = 'kx349291 |cyanophage s-rim44 isolate es_42_0910| complete genome|complete|usa|synechococcus|cyanophage'

#Fix other cyanophage anomalies

c = 0
l = []
for i in desc:
    if i[-1] == '|':
        l.append(c)
    c+=1
    
for i in l:
    desc[i] = desc[i] + 'cyanophage' 

In [202]:
#Species name missing detection
c = 0
l = []
for i in desc:
    if i[-1] == '|':
        print(i, c)
    c+=1

In [203]:
for i in l:
    print(desc[i])

In [204]:
#load sequence id and description into dataframe

df = pd.DataFrame({'Sequence description' : desc})
df['ID'] = eyed

In [205]:
#Get species name by taking last '|' split section from description

l = [x.split('|')[-1].lower() for x in df['Sequence description']]
df['Species'] = l

In [206]:
#New dataframe with only unique species names

df2 = df.drop_duplicates(subset = 'Species')

#leaves 17046 rows

In [207]:
#Shorten species name upto 'virus' for viruses and upto 'phage' for phages

tl = list(df2['Species'])
tl2 = []

for i in tl:
    if 'virus' in i:
        tl2.append(i.split('virus')[0] + 'virus')
    elif 'phage' in i:
        tl2.append(i.split('phage')[0] + 'phage')
    else:
        tl2.append(i)
        
df2.loc[:, 'Species'] = tl2

c:\users\michael\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [208]:
#checking which entries have species names starting with phage or virus

# c = 0
# for i in tl2:
#     if i == 'virus':
#         print(tl[c])
#     elif i == "phage":
#         print(tl[c])
#     c+=1

phage nbeco005
phage nbsal001
phage nbsal005
phage nbeco002
phage nbeco004
phage nbeco003
phage nbeco001
phage nbsal002
phage nbsal004
phage nbsal003
phage apr34_1792
phage ys1-2_2434
phage fako05_000032f
phage fako27_000238f
phage fako27_000271f
phage apr34_1784
phage apr34_1789
virus sp. ctgox21
phage altai3
phage nc-b
phage nc-g
phage nc-a
phage ngi136
phage ng54
phage nv18
phage nv21
phage ng55
phage ngi4
phage 5p_2
phage 5p_3
phage 5p_1
phage pbkp05
virus sp.
phage 206
phage 203
phage medpe-swcel-c56
phage nctb


In [209]:
#if you want to look at the the weridest species names

# for i in df2['Species'].tolist():
#     if 'virus' not in i and 'phage' not in i and 'viroid' not in i and 'viridae' not in i and 'satellite' not in i:
#         print(i)

synchytrium chytrid fungus meld element
possum hepacivrus
picornavirales sp.
caudovirales sp. ctown3
reptile-associated circular dna molecule
insect-associated ssdna molecule
caudovirales gx15bay
pseudomonas aeruginosa hage pazq-1
circular genetic element sp.
avian hdv-like agent
caudovirales sp.
hyposidra talaca npv
gokushovirinae environmental samples
chimpanzee faeces associated circular dna molecule 1
bovine faeces associated circular dna molecule 1
porcine faeces associated circular dna molecule-1
lake sarah-associated circular molecule 3
lake sarah-associated circular molecule 5
lake sarah-associated circular molecule 8
lake sarah-associated circular molecule 12
lake sarah-associated circular molecule 10
lake sarah-associated circular molecule 11
lake sarah-associated circular molecule 4
lake sarah-associated circular molecule 6
lake sarah-associated circular molecule 7
lake sarah-associated circular molecule 1
lake sarah-associated circular molecule 9
lake sarah-associated circu

In [210]:
#Drop duplicates on Species again

df2 = df2.drop_duplicates(subset = 'Species').reset_index(drop= True)

#remove just 'virus' at index 688
#100% make sure this code does not run twice
df2 = df2.drop(axis = 0, index = 688).reset_index(drop= True)

#leaves 4818 rows

#Making new 'Genome type' column

df2['Genome type'] = 'dunno'

End of primary filtering for species in bulk sequence data file
Start of genome type labelling

Using ViRAD .txt files 

In [211]:
#loading files into lists p, n and d

f = open('plus ssRNA.txt', 'r')
p = f.readlines()
f.close()
f = open('minus ssRNA.txt', 'r')
n = f.readlines()
f.close()
f = open('dsRNA.txt', 'r')
d = f.readlines()
f.close()

In [212]:
#join p, n and d into three large strings

p2 = ''
p2 = p2.join(p).lower()
n2 = ''
n2 = n2.join(n).lower()
d2 = ''
d2 = d2.join(d).lower()

Anomaly detection

In [213]:
#found both in p2 and n2

for i in df2['Species'].tolist():
    if i in p2 and i in n2:
        print('wtf', i, df2['Species'].tolist().index(i))

##Reveals 3 anomalies. output: wtf cosavirus 174 \n wtf virus 688 \n wtf  2403
#Hardcoded species name to species name missing values
#Cosavirus is probably plus ssRNA. Manually annotated in cell below
#hardcoded removal of just 'virus' from df2 in cell above

wtf cosavirus 174


In [232]:
#found both in p2 and d2

for i in df2['Species'].tolist():
    if i in p2 and i in d2:
        print('wtf', i, df2['Species'].tolist().index(i))
        
        
#gives output wtf pseudomonas phage 27 \n wtf phage 49 \n wtf rna virus 305
#d2 has 5 pseudomonas phage strains and p2 has 2. Hoping to manually hardcode them below
#rna virus comes from 'rna virus hoopoe/bbanka01/2015/hun' which has unknown genome. hardlabelling
df2.loc[df2['Species'] == 'rna virus', 'Species'] = 'rna virus hoopoe'
df2.loc[df2['Species'] == 'rna virus hoopoe', 'Genome type'] = 'unknown RNA'

# phage 49 will be hard to fix for reasons hard to explain rn. Will ignore for now.

wtf pseudomonas phage 27
wtf phage 49
wtf rna virus 305


In [215]:
#found both in n2 and d2

for i in df2['Species'].tolist():
    if i in n2 and i in d2:
        print('wtf', i, df2['Species'].tolist().index(i))
        
        
#gives output: wtf yam virus 1017
#yam virus y is probably an unknown RNA virus. Skimming the paper did not reveal genome type. The family Betaflexiviridae is
# probably a plus RNA family, so manually labelling as plus RNA

wtf yam virus 1017


In [239]:
#label

c = 0 

for i in df2['Species'].tolist():
    if i in p2:
        df2.loc[c, 'Genome type'] = 'plus ssRNA'
    elif i in n2:
        df2.loc[c, 'Genome type'] = 'minus ssRNA'
    elif i in d2:
        df2.loc[c, 'Genome type'] = 'dsRNA'
    c+=1
                
##Reveals 3 anomalies. output: wtf cosavirus 174 | wtf virus 688 | wtf  2403
#Hardcoded 2403 species name
#Cosavirus is probably 'plus ssRNA'. Manually annotating
df2.loc[174, 'Genome type'] = 'plus ssRNA'
#hardcoded removal of just virus from df2 in cell above
#hardcoding 'yam virus' as 'plus ssRNA' 
df2.loc[1017, 'Genome type'] = 'plus ssRNA'

In [303]:
#hardcoding 5 + 2 = 7 pseudomonas phages

df3 = df.drop_duplicates(subset= 'Species').reset_index(drop = True)

pl = []
c = 0
for i in tl2:
    if i == 'pseudomonas phage':
        pl.append(c)
    c+=1
        
for i in pl:
    if df3.loc[i, 'Species'] in p2:
        nd = dict(df3.loc[i, :])
        nd['Genome type'] = 'plus ssRNA'
        df2 = df2.append(nd, ignore_index=True)
    elif df3.loc[i, 'Species'] in d2:
        nd = dict(df3.loc[i, :])
        nd['Genome type'] = 'dsRNA'
        df2 = df2.append(nd, ignore_index=True)
        
#this adds 1 new plus ssRNA value and 2 new dsRNA values

In [377]:
df2.loc[df2['Genome type'] == 'minus ssRNA']

,Sequence description,ID,Species,Genome type
19,mh169353 |influenza a virus (a/waterfowl/korea...,MH169353,influenza a virus,minus ssRNA
248,mn550995 |melon yellow spot virus strain zheji...,MN550995,melon yellow spot virus,minus ssRNA
313,mn258361 |marburg marburgvirus isolate predict...,MN258361,marburg marburgvirus,minus ssRNA
355,mn567046 |university of helsinki virus-2 isola...,MN567046,university of helsinki virus,minus ssRNA
423,mn729604 |avian metapneumovirus isolate hungar...,MN729604,avian metapneumovirus,minus ssRNA
589,mn745085 |human metapneumovirus isolate bj0077...,MN745085,human metapneumovirus,minus ssRNA
608,mn038329 |viral hemorrhagic septicemia virus i...,MN038329,viral hemorrhagic septicemia virus,minus ssRNA
624,mk251241 |european bat lyssavirus isolate 5220...,MK251241,european bat lyssavirus,minus ssRNA
1109,mh428628 |metapneumovirus sp. isolate ug-ngo-a...,MH428628,metapneumovirus,minus ssRNA
1680,mk028835 |bundibugyo ebolavirus isolate ebola ...,MK028835,bundibugyo ebolavirus,minus ssRNA


In [293]:
len(df2.loc[df2['Genome type'] == 'dsRNA'])

83

Code above labels 579 rows as 'plus ssRNA' and 39 rows as 'minus ssRNA' and 85 as 'dsRNA' and 1 as 'unknown RNA'

4116 rows left unlabelled

End of using the three ViRAD files.

Data could be labelled by matching the family/order names in those files. Will explore later

Starting the use of NCBI taxonomy and ICTV file

In [320]:
#Loading data

df3 = pd.read_csv('tax.csv')
df4 = pd.read_csv('ICTV 2018b.v2.csv')

In [321]:
#Take first comma separated part of 'Selected lineage' column as a 'Fam' column

df3['Fam'] = [x.split(',')[0] for x in df3['Selected lineage'].to_list()]

In [328]:
#Make a new dataframe df5 which relates a Family with its Genome type

df5 = pd.DataFrame({'Fam' : df4['Family'], 'Genome type' : df4['Genome Composition']})
df5 = df5.drop_duplicates(subset = 'Fam')

#merge df3 to df5 to get an expected Genome type for every NCBI entry

df3 = pd.merge(df3, df5, how = 'left', on = 'Fam')

#Out of 212830 entries, only 4028 remain unlabelled
#Only 15008 out of 212k entries have a unique taxonomy name

In [341]:
# for some reason 'Neighbor' in df3 and 'ID' in df have no common entries.
# downloading data from ncbi (not ncbi virus) , that share the same accession number scheme

15008

In [340]:
df3['Taxonomy name'] = df3['Taxonomy name'].lower()

,Representative,Neighbor,Host,Selected lineage,Taxonomy name,Segment name,Fam,Genome type
0,NC_003663,HQ420896,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment,Poxviridae,dsDNA
1,NC_003663,KY463519,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment,Poxviridae,dsDNA
2,NC_003663,HQ420897,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment,Poxviridae,dsDNA
3,NC_003663,MK035759,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment,Poxviridae,dsDNA
4,NC_003663,KY569019,"human,vertebrates","Poxviridae,Orthopoxvirus,Cowpox virus",Cowpox virus,segment,Poxviridae,dsDNA


In [364]:
'Y15034' in df3['Representative']

False

In [366]:
df3.sort_values(by='Neighbor')

,Representative,Neighbor,Host,Selected lineage,Taxonomy name,Segment name,Fam,Genome type
34557,NC_001802,A04321,"human,vertebrates","Retroviridae,Lentivirus,Human immunodeficiency...",Human immunodeficiency virus 1,segment,Retroviridae,ssRNA-RT
33594,NC_001802,A07108,"human,vertebrates","Retroviridae,Lentivirus,Human immunodeficiency...",Human immunodeficiency virus 1,segment,Retroviridae,ssRNA-RT
34896,NC_001802,A07116,"human,vertebrates","Retroviridae,Lentivirus,Human immunodeficiency...",Human immunodeficiency virus 1,segment,Retroviridae,ssRNA-RT
35727,NC_001802,A07867,"human,vertebrates","Retroviridae,Lentivirus,Human immunodeficiency...",Human immunodeficiency virus 1,segment,Retroviridae,ssRNA-RT
45577,NC_001616,A08776,land plants,"Potyviridae,Potyvirus,Potato virus Y",Potato virus Y,segment,Potyviridae,ssRNA(+)
...,...,...,...,...,...,...,...,...
48784,NC_002602,Z96099,land plants,Cucumber mosaic virus satellite RNA,Cucumber mosaic virus satellite RNA,segment,Cucumber mosaic virus satellite RNA,NaN
47467,NC_003811,Z97357,land plants,"Virgaviridae,Tobravirus,Tobacco rattle virus",Tobacco rattle virus,segment RNA 2,Virgaviridae,ssRNA(+)
150200,NC_003520,Z97873,land plants,"Virgaviridae,Pomovirus,Beet soil-borne virus",Beet soil-borne virus,segment RNA 1,Virgaviridae,ssRNA(+)
169589,NC_003855,Z98197,land plants,Peanut stunt virus satellite RNA,Peanut stunt virus satellite RNA,segment,Peanut stunt virus satellite RNA,NaN


In [374]:
'A04321' in df['ID'].to_list()

False

In [347]:
len(df3.loc[df3['Genome type'].isnull()])

4028

In [389]:
ndf = pd.read_csv('first test label.csv')
ndf = ndf[~ndf['Genome type'].isin(['dunno'])].reset_index(drop=True)
ndf = ndf.drop(axis = 1, labels = 'Unnamed: 0')

In [391]:
l = []

for i in ndf['Sequence description']:
    l.append(desc.index(i))

In [399]:
c = 0
for seq_record in SeqIO.parse('Complete NCBI Virus Genbank only.fasta', "fasta"):
    if c in l:
        l.append(seq_record.seq)
    c+=1

In [412]:
import numpy as np
a = np.array(l)
a = a.reshape(704, 2, order = 'F')

In [417]:
ndf['Seq'] = l[704:]

In [424]:
ndf.to_csv('first test label.csv', index = False)

In [433]:
l[-5]

Seq('CACAAAATATAAGAAGGAAAGCTGGATTTCCTACCTTTGTGTTTCCATTGTCGA...AAT', SingleLetterAlphabet())

In [435]:
str(l[-5].translate(11))

'HKI*EGKLDFLPLCFHCRSIGQTLFDASSAYFHHQISETRVLHWF*GP*SCS*PRRDRYSL*SGI*LQR*VRLFGESPQPTV*RRRS*ELFCLL*QWS*RLLRPTTPCLVRDCVPGVCYLLFLCLGTRGSGRRGWSSFEEVRSL*KRESVSAGCAARNRRGILLFVSKESLPRMYHPSCKGYVLLLANMASFAILVDTFLMGVHNPSHVVPVVALMTLCATPLWLSASYVIYLLRYMLVRLSPEAKQDRVYYTATTAPYFDPALGVMMQFAPSSGGPSIEVQINPSWISLLGSSLKLNVGDASNESAVLGSFYSPVKAGDEPASLVAIKSGPTTIGFGCRTKIDGEDCLLTAHHVWCNSMRPTGLAKAGKQVSVEDWEISMSSSDKMLDFAIVRVPTHVWSKLGVKSTPLVCPSSKDVITCYGGSSSDCLMSGVGSSSTSEFTWKLTHTCPTAAGWSGTPLYSSRGVVGMHVGFEEIGKLNRGVNMFYVANYLLRSNETLPPELSIIEIPFDDVETRSYEFIEVEIKGRGKAKLGKREFAWIPESGKYWADEDEDELPPPPKLVGGKLVWENAQETVAVENLNCQRAAGSRPLPPSLNLHATSSAKEKSPLPKACHSDLLGERLASLESCVEKILQKMSCEQYLPSQSSQTTLGQSEAPKLSLAPCYYKQESLILPESQQTLKELVKTSLSATPPPNPVAALDQKPGPSTQSLKKSVRRRNRRKSTKKPAQGSPSPASPPPTKTS*RGICNLLLSVLLKDSSYLARRQICIT*VPLSWLS*GIAIQCASLLSRSPTPPER*ERVDSA*FHQSRWLIS*LSACSSGRKISLRSLSGREFRLSLGWACHLIGKLAACLAT*ELNTLVALLLRLTSRVLTGLCKTGSCGLMLR*E*F*AALARHWLGLLGIGFRVS*TQSSSYLTVH*LSNCSRE**SLDLTAPPQPIAEFVALWLSSLAPHGVLLWVMILLKDGLMAQRTST*G*AMFARITSLAPRLLKA